# Feedforward Neural Networks — Training and Evaluating the Model on Forex Data
a medium.com article

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
# from tensorflow.keras.models import Sequential
from keras.src.models import Sequential
# from tensorflow.keras.layers import Dense
from keras.src.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
data = pd.read_csv("../src/XAGUSD-197001010000--H1-rates.csv", sep=r'\s+', engine='python')  # Handles space-delimited files
data = data.dropna()

In [ ]:
# Parse date and time
data['DATETIME'] = pd.to_datetime(data['<DATE>'] + ' ' + data['<TIME>'])
data.set_index('DATETIME', inplace=True)

In [ ]:
# Drop old date/time columns
data.drop(columns=['<DATE>', '<TIME>'], inplace=True)

In [ ]:
# Rename columns for clarity
data.columns = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL', 'VOL', 'SPREAD']

# Drop any rows with missing values (just in case)
data = data.dropna()

# Scale features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

In [ ]:
# Create features (X) and target (y) — predict next CLOSE based on current row
X = scaled_data[:-1]
y = scaled_data[1:, 3]  # column index 3 = 'CLOSE'
y = y.reshape(-1, 1)  # reshape for Keras

In [ ]:
# Train/test split
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2, verbose=1)

In [ ]:
# Predict and inverse scale
y_pred = model.predict(X_test)
# Combine back for inverse transformation
dummy = np.zeros((len(y_pred), scaled_data.shape[1]))
dummy[:, 3] = y_pred.flatten()  # set predicted CLOSE
y_pred_inv = scaler.inverse_transform(dummy)[:, 3]

In [ ]:
dummy[:, 3] = y_test.flatten()  # set true CLOSE
y_test_inv = scaler.inverse_transform(dummy)[:, 3]

In [ ]:
# Calculate MSE
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f"Mean Squared Error: {mse}")